LSM EXP 07_Aiffel
<br/>**7. 뉴스 요약봇 만들기**

**개발 환경**
<br/>**데이터 정보**
<br/>**데이터 수집**
<br/>**텍스트 정규화**
<br/><br/>**추상적 요약**
- 텍스트 정규화
> 불용어 사전
<br/>Set 변환(자료형 시간복잡도 계산)
<br/>축약어 사전
<br/>멀티 프로세싱

- train, test 데이터 분리
- 어텐션 메커니즘
- 실제 결과와 요약문

**추출적 요약**
* summa

**결론**
<br/>**참고문헌**

#머리말

#개발 환경

In [ ]:
pip install importlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for importlib: filename=importlib-1.0.4-py3-none-any.whl size=5875 sha256=53818d4c66a7dc79a74f5af8ff2b36ac52be73bf0c60f0325bd84f1dc7d35d1f
  Stored in directory: /root/.cache/pip/wheels/86/e4/cb/62b0e9efd7da1e984baec0c0ded0b727a7ed25e1904ed51fca
Successfully built importlib


In [ ]:
pip install importlib-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 592 kB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54412 sha256=0a1236d10e113b7c471276525c9de284a5f2527afa2deb9df8965a8e7272505b
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [ ]:
import nltk
import summa
import re
from bs4 import BeautifulSoup

**NLTK(Natural Language Toolkit)**는 교육용으로 개발된 자연어 처리 및 문서 분석용 파이썬 패키지다.
<br/>말뭉치, 토큰 생성, 형태소 분석, 품사 태깅의 기능을 사용한다.

**summa**는 텍스트 요약과 키워드 추출을 실행하는 TextRank 라이브러리이다.
<br/>TextRank는 구글의 페이지 랭크 알고리즘을 기반하여
<br/>텍스트로 이루어진 글에서 특정 단어가 다른 문장과 얼마만큼의 관계를 맺고 있는지를 계산하는 알고리즘이다.

**re(regex)**는 특정 문자 또는 문자열이 존재하는지나 어느 위치에 있는지와 같은 기능을 제공하는 정규표현식 라이브러리이다.

**BeautifulSoup**은 HTML 및 XML 파일에서 데이터를 Parsing 할 수 있는 라이브러리이다.<br/>Parsing이란 어떤 페이지(문서, html 등)에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출해 가공하는 것이다.

In [ ]:
from importlib_metadata import version
import tensorflow
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

**importlib**은 이미 임포트 해둔 모듈의 내용을 수정하고 싶을 때 사용한다.
<br/>임포트를 다시 하지 않고 모듈을 개발자의 환경에 맞게 수정한다.
<br/>코드 편집기에 재접속했을 때도 그 개발 환경 변경 사항이 유지된다.

**importlib_metadata**은 pip와 pkg_resources으로 패키지 메타데이터에 접근한다.
<br/>pkg_resources의 entry point API(프로그램의 진입점)와 metadata API 기능을 담당한다.
<br/>메타데이터는 package>-version-py version.egg-info 파일에 저장된다.





**tensorflow**는 구글이 개발한 오픈소스 소프트웨어 딥러닝 및 머신러닝 라이브러리이다.
<br/>수학 계산식과 데이터의 흐름을 노드와 엣지를 사용한 방향성 그래프, 데이터 플로우 그래프로 나타낸다.

**pandas**는 행과 열로 이루어진 데이터 객체를 만들어 안정적으로 대용량의 데이터들을 처리한다.

**numpy**는 array 단위로 벡터와 행렬을 계산한다.

**os(Operating System)**는 운영체제에서 제공되는 여러 기능을 파이썬에서 수행한다. <br/>예를 들어, 파일 복사, 디렉터리 생성, 파일 목록을 구할 수 있다.

**warnings**는 실행시에 경고메시지를 지우는 역할을 한다.

In [ ]:
import time
from time import strftime
import sys
import collections

**time**은 epoch time(Unix time, POSIX time)을 다룬다.
<br/>epoch time(timestamp)은 UTC(GMT+0) 기준으로 1970년 1월 1일 0시 0분 0초부터의 경과 시간을 나타낸다.

**time.strftime**은 '연-월-일' 등 원하는 날짜/시간 포맷으로 출력한다.

**sys**는 파이썬 인터프리터가 제공하는 변수와 함수를 직접 제어한다.
<br/>명령 행에서 인수 전달(sys.argv), 강제로 스크립트 종료(sys.exit), 자신이 만든 모듈 불러와 사용(sys.path)하기 등의 기능이 있다.

**collections**는 자료구조 라이브러리이다.
<br/>스택과 큐 구현(deque), 등장 회수(counter) 세기 등의 기능이 있다.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 23 20:37:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

할당된 GPU를 확인한다.

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


고용량 메모리 VM에 액세스한다.

#데이터 정보

[news_summary_more.csv](https://github.com/sunnysai12345/News_Summary)

데이터셋 제작자는 채팅 문맥을 분석하고 이해하는 일을 하고 있는 개발자다.
<br/>BeautifulSoup의 bs4 모듈을 이용해 웹크롤링을 했다.
<br/>뉴스 기사의 문법과 어휘는 좋은 통찰력을 준다고 믿는다.

2017년 Hindu(인도 제2위 영문 일간지), The Times of India(인도 최대 영문 일간지)
<br/>Guardian(영국 일간지)의 기사와 Inshorts(요약문)를 수집한 데이터이다.
<br/>Author_name, Headlines, Url of Article, Short text, Complete Article의 정보를 포함한다.
<br/>텍스트 요약(Text Summarization) 도구를 만드는 데 사용된다.

# 데이터 수집

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

**urllib.request**은 다이제스트 인증, 리디렉션, 쿠키 등과 같은 URL이나 HTTP를 여는 데 도움이 되는 함수와 클래스를 정의한다.
<br/>웹 페이지 요청 및 데이터를 가져온다.

**iso-8859-**1는 아스키 코드로 인코딩한다.
<br/>아스키 코드는 영어를 표현하는 문자열 인코딩 방식이다.

In [ ]:
data.sample(10)

,headlines,text
88949,"Child cries as F1 racer crashes out, meets dri...","A six-year-old F1 fan, who cried in the stands..."
32168,Cops to protect Hyd mosque after Shiva idol in...,Police were assigned to protect the Alijapur m...
5490,"Kirsten, Gibbs, Powar to appear for India wome...",India's 2011 World Cup-winning coach Gary Kirs...
91678,Ikea responds to Balenciaga's Ã¢ÂÂ¹1 lakh loo...,Furniture retailer company Ikea has responded ...
79475,"Ã¢ÂÂ¹19,000 crore black money found in HSBC, ...",Finance Minister Arun Jaitley has said the Inc...
36435,"ED traces Ã¢ÂÂ¹5,000cr moved through Nirav, C...",The Enforcement Directorate has traced over Ã¢...
35634,Koena Mitra slams Sonam for her tweet on Kathu...,Actress Koena Mitra has slammed Sonam Kapoor f...
46842,WhatsApp rolls out group description feature f...,Facebook-owned messaging service WhatsApp has ...
76532,"Couple buys entire street in wealthy area, ang...",A couple has bought a street in the wealthy ne...
54900,Mumbai to get its biggest sports complex in Da...,The Brihanmumbai Municipal Corporation's garde...


text와 headlines 정보로 구성되어 있다.

In [ ]:
print('전체 샘플수 :', (len(data)))

전체 샘플수 : 98401


전체 98401개의 샘플이 있다.

In [ ]:
print('headlines 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())
print('text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())

headlines 열에서 중복을 배제한 유일한 샘플의 수 : 98280
text 열에서 중복을 배제한 유일한 샘플의 수 : 98360


In [ ]:
data.drop_duplicates(subset = ['headlines'], inplace=True)
print('전체 샘플수 :', (len(data)))

전체 샘플수 : 98280


데이터프레임의 **drop_duplicates()**를 사용하여 중복 샘플을 제거한다.

중복이 제거되면서 샘플 수가 98,280개로 줄어들었다.

In [ ]:
print(data.isnull().sum())

headlines    0
text         0
dtype: int64


**.isnull().sum()**을 사용하여 데이터프레임에 Null 값이 있는지  확인한다.

추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습할 수 있어요.
<br/>추출적 요약을 하는 경우에는 오직 text열만을 사용하세요.

#추상적 요약

##텍스트 정규화

**텍스트 정규화(text normalization)**는 추가적으로 데이터를 처리하려고 자연어로 된 텍스트를 준비하는 과정이다.

텍스트 정규화에는 여러 방법이 있다.


**토큰화(tokenization)**
<br/>NLTK 라이브러리에서 토큰화를 한다.
* word_tokenize(단어 토크나이저)
* sent_tokenize(문장 토크나이저)
* regexp_tokenize(정규 표현식 기반의 토크나이저)
* WordPunctTokenizer(이모티콘을 이용한 감성 분석)

**단어의 대·소문자 통일**
<br/>**불용어 제거**
<br/>**형태소 분석(stemming)**
* Porter(포터)
* Lancaster(랭커스터)
* lemmatize(원형 추출)

뉴스 텍스트 요약 도구에 맞는 텍스트 정규화 방법을 사용한다.

###불용어 사전

불용어는 큰 의미가 없다.
<br/>자주 등장하지만 텍스트 이해에 기여하는 바가 없기 때문이다.
<br/>

NLTK 라이브러리 자체 내에 정의된 영어 불용어 사전이 있다.


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

In [ ]:
for i in range(len(nltk_stopwords)):
  if i % 17 == 0 and i != 0:
    print()
    print(nltk_stopwords[i-17:i], end='')
print() 
print(nltk_stopwords[-9:])  

print("\nnltk 정규화 사전 단어의 수: ", len(nltk_stopwords))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves']
['he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs']
['themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been']
['being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because']
['as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above']
['below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there']
['when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not']
['only', 'own', 'same', 'so', '

###Set 변환(자료형 시간복잡도 계산)

In [ ]:
nltk_stopwords = set(stopwords.words('english'))

불용어 사전 list를 set으로 바꾼다.

Python은 list보다 set으로 찾는 것이 훨씬 작업속도가 빠르다.
<br/>

In [ ]:
#import time
#import sys, collections
#from time import strftime
list_process = []
start_time = time.time()
for i in range(1, 100000):
    list_process.append(i)

list_dq = collections.deque(list_process)
list_d = list_dq.reverse()

print(time.time() - start_time)

0.014227151870727539


In [ ]:
set_process = set()
set_adder = list(range(1, 100000))
start_time = time.time()
set_process.update(set_adder)
set_dq = collections.deque(set_process)

set_d = set_dq.reverse()
print(time.time() - start_time)

0.004404306411743164


list에서 in 연산자를 사용하면 사실 상 for문을 한번 도는 것이기 때문에 선형 시간인 O(n)의 시간 복잡도를 가진다.
<br/>list에서 set으로 바꾸고 in 연산을 한다면 선형 시간 O(n)에서 O(1)로 작업 시간을 줄일 수 있다. 

Python의 set과 dict는 이진 트리가 아니라 해시 테이블로 구현되어 있어서 평균 O(1)의 시간이 걸린다.

Text 전처리 시에서만 호출하고 이미 상대적으로 문장 길이가 짧은 Summary 전처리할 때는 호출하지 않을 예정이다.

###축약어 사전

[스택오버플로우](https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python#)의 답변에 올라온 사용자 정의 축약어 사전을 저장한다.

In [ ]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("스택오버플로우 정규화 사전 단어의 수: ", len(contractions))

스택오버플로우 정규화 사전 단어의 수:  120


In [ ]:
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower()
    sentence = BeautifulSoup(sentence, "lxml").text
    sentence = re.sub(r'\([^)]*\)', '', sentence) 
    sentence = re.sub('"','', sentence) 
    sentence = re.sub(r"'s\b","", sentence) 
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    sentence = re.sub('[m]{2,}', 'mm', sentence) 

    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in tokens]
    return(' '.join(stemming_words))

데이터 전처리 함수 preprocess_sentence를 정의한다.

**lower()**는 텍스트를 소문자화한다.

**BeautifulSoup**에서  br/, a href = ... 등의 html 태그 제거를 한다.

**re**에서 괄호로 닫힌 문자열 (...), 쌍따옴표, 소유격을 제거하고
<br/>약어 정규화, 영어 외 문자(숫자, 특수문자 등) 공백으로 변환, m이 3개 이상이면 2개로 변경한다.

If 문에서 **remove_stopwords=True**면 불용어를 제거하고, **False**면 제거하지 않는다. 

**SnowballStemmer**는 Porter Stemmer의 에러를 수정한 어간 추출 알고리즘이다.
<br/>어간 추출은 단어를 어간 형태로 표현함으로써 단어의 개수를 줄이는 기법이다.
<br/>[Snowball 공식 홈페이지](https://snowballstem.org/algorithms/)에 정의된 어간 사전을 바탕으로 하여 어간을 추출한다.
<br/>예를 들어 see, saw, seen 는 모두 "see"로 표기를 통일한다.
<br/>Stemming을 통하여 단어 각각이 전하는 핵심 뜻을 간직하는 동시에, 복잡도를 낮춘다.

In [ ]:
temp_text = 'Everything I bought was great, infact I ordered twice and the third ordered was<br />for my mother and father.'
temp_summary = 'Great way to start (or finish) the day!!!'

print("text: ", preprocess_sentence(temp_text))
print("summary:", preprocess_sentence(temp_summary, False))  

text:  everything bought great infact ordered twice third ordered wasfor mother father
summary: great way to start the day


###멀티 프로세싱

In [ ]:
from multiprocessing import Pool
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
 
    workers = kwargs.pop('workers')
    pool = Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

키워드 항목 중 workers 파라메터를 꺼낸다.
<br/>위에서 가져온 workers 수로 프로세스 풀을 정의
<br/>실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
<br/>작업 결과를 합쳐서 반환

전처리 이전과 이후의 문장을 비교한다.

추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습한다.

In [ ]:
%time clean_train_reviews = apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)  

In [ ]:
clean_text = []

for s in data['text']:
    clean_text.append(preprocess_sentence(s))

print("text 전처리 후 결과: ", clean_text[:5])

text에 대해서 전처리 함수를 호출해 줄 때는, 불용어 제거를 수행한다.

In [ ]:
clean_headlines = []

for s in data['headlines']:
    clean_headlines.append(preprocess_sentence(s, False))

print("headlines 전처리 후 결과: ", clean_headlines[:5])

headlines에 대해서 전처리 함수를 호출해 줄 때는, 불용어 제거를 수행하지 않는다.

In [ ]:
data['Text'] = clean_text
data['Headlines'] =  clean_headlines

data.replace('', np.nan, inplace=True)

데이터들을 데이터프레임에 저장하고
<br/>빈(empty) 값을 가진 샘플들이 있다면, Null 값을 가진 샘플로 대체한다.

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(axis=0, inplace=True)

.isnull().sum()을 사용해서 Null 값이 생겼는지 확인한다.<br/>Headlines 열에서 70개의 Null 값이 생겼다.<br/>이 샘플들을 제거한다.

In [ ]:
print('전체 샘플수 :', (len(data)))

전체 샘플 수는 

##train, test 데이터 분리

In [ ]:
# 길이 분포 출력
import matplotlib.pyplot as plt

text_len = [len(s.split()) for s in data['Text']]
summary_len = [len(s.split()) for s in data['Headlines']]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))

plt.subplot(1,2,1)
plt.boxplot(text_len)
plt.title('Text')
plt.subplot(1,2,2)
plt.boxplot(summary_len)
plt.title('Summary')
plt.tight_layout()
plt.show()

plt.title('Text')
plt.hist(text_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

plt.title('Summary')
plt.hist(summary_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

##어텐션 메커니즘

일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

##실제 결과와 요약문

원래의 요약문(headlines 열)과 학습을 통해 얻은 추상적 요약의 결과를 비교해 보세요

#추출적 요약

추출적 요약을 하는 경우에는 text열만을 사용한다.

추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

Summa의 summarize를 사용하여 추출적 요약을 해보세요.

##Summa